# Bulldozer Regression

This notebook will be recreating `end-to-end-bulldozer-price-regression` notebook with the help of pipeline and columnstransformer for easier preprocessing process

In [73]:
# Importing Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Splitting data
from sklearn.model_selection import train_test_split

# Preprocessing
from category_encoders.binary import BinaryEncoder
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelling
from sklearn.ensemble import RandomForestRegressor

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Evalutaion
from sklearn.metrics import root_mean_squared_log_error


# Load Dataset

In [2]:
data_dic = pd.read_excel("data/Data Dictionary.xlsx")
data_dic

d:\miniconda_env\env\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Variable,Description,Unnamed: 2
0,SalesID,unique identifier of a particular sale of a ...,NaN
1,MachineID,identifier for a particular machine; machin...,NaN
2,ModelID,identifier for a unique machine model (i.e. ...,NaN
3,datasource,source of the sale record; some sources are...,NaN
4,auctioneerID,"identifier of a particular auctioneer, i.e. ...",NaN
5,YearMade,year of manufacturer of the Machine,NaN
6,MachineHoursCurrentMeter,current usage of the machine in hours at tim...,NaN
7,UsageBand,"value (low, medium, high) calculated compari...",NaN
8,Saledate,time of sale,NaN
9,Saleprice,cost of sale in USD,NaN


In [67]:
train = pd.read_csv("data/TrainAndValid.csv",parse_dates=["saledate"])
test = pd.read_csv("data/Valid.csv",parse_dates=["saledate"])
test_solution = pd.read_csv("data/ValidSolution.csv")

C:\Users\berna\AppData\Local\Temp\ipykernel_15172\3322668567.py:1: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("data/TrainAndValid.csv",parse_dates=["saledate"])


# EDA

We will be using all the features, and since the focus of this project is to practice building machine learning model with pipeline and columntransformer, we will only do a short EDA section and we will try to define what to do for each features

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   412698 non-null  int64         
 1   SalePrice                 412698 non-null  float64       
 2   MachineID                 412698 non-null  int64         
 3   ModelID                   412698 non-null  int64         
 4   datasource                412698 non-null  int64         
 5   auctioneerID              392562 non-null  float64       
 6   YearMade                  412698 non-null  int64         
 7   MachineHoursCurrentMeter  147504 non-null  float64       
 8   UsageBand                 73670 non-null   object        
 9   saledate                  412698 non-null  datetime64[ns]
 10  fiModelDesc               412698 non-null  object        
 11  fiBaseModel               412698 non-null  object        
 12  fi

In [5]:
train.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# order the data based on saledate
train = train.sort_values("saledate")
train.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
205615,1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
274835,1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
141296,1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
212552,1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
62755,1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,Lever,NaN,NaN


In [7]:
# make a copy of the dataset
train_ori = train.copy()

# Defining X and y

In [8]:
X_train = train.drop("SalePrice",axis=1)
y_train = train["SalePrice"]

# Preprocessing

we will:
- Saledate:
    - add SaleYear
    - add SaleMonth
    - add SaleDay
    - add SaleDayOfWeek
    - add SalDayOfYear
    - Remove Saledate

- numerical columns: 
    - add a new boolean column `x_is_missing`
    - fill missing values with median
    - scale them with robust_scaler

- categorical columns:
    - add new boolean column `X_is_missing`
    - fill missing values with "unknown"
    - encode them 
        - Ordinal : UsageBand, ProductSize
        - Binary : the rest of categorical column

In [9]:
X_train.columns

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'saledate',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls'],
      dtype='object')

## SaleDate

Create a new class to create new feature

In [10]:
class NewFeature(BaseEstimator,TransformerMixin):
    def __init__(self):
        self.missing_colum = []

    def fit(self, X, y=None):
        for column in X.columns:
            if X[column].isna().sum():
                self.missing_colum.append(column)
        return self
        
    def transform(self, X, y=None):
        X_copy = X.copy()

        # for SaleDate
        X_copy["SaleYear"] = X_copy["saledate"].dt.year
        X_copy["SaleMonth"] = X_copy["saledate"].dt.month
        X_copy["SaleDay"] = X_copy["saledate"].dt.day
        X_copy["SaleDayOfWeek"] = X_copy["saledate"].dt.dayofweek
        X_copy["SaleDayOfYear"] = X_copy["saledate"].dt.dayofyear


        # for missing columns
        for column in self.missing_colum:
            X_copy[f"{column}_is_missing"] = X_copy[column].isna()

        X_copy = X_copy.drop("saledate",axis=1)    
        return X_copy

# Pipeline

In [11]:
numerical_columns = X_train.select_dtypes("number").columns.to_list()
categorical_binary_column = X_train.select_dtypes("O").columns.to_list()
categorical_binary_column.remove("UsageBand")
categorical_binary_column.remove("ProductSize")
categorical_ordinal_column = ["UsageBand","ProductSize"]

In [12]:
numerical_pipeline = Pipeline([
    ("median_imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler()) 
])

categorical_binary_pipeline = Pipeline([
    ("unknown_imputer", SimpleImputer(strategy="constant",fill_value="unknown")),
    ("encoder", BinaryEncoder()),
])

categorical_ordinal_pipeline = Pipeline([
    ("unknown_imputer", SimpleImputer(strategy="constant",fill_value="unknown")),
    ("encoder", OrdinalEncoder())
])

transformer = ColumnTransformer([
    ("numerical" , numerical_pipeline, numerical_columns),
    ("categorical_binary", categorical_binary_pipeline,categorical_binary_column),
    ("categorical_ordinal", categorical_ordinal_pipeline,categorical_ordinal_column),
], remainder="passthrough")

In [19]:
full_pipeline = Pipeline([
    ("feature_engineering", NewFeature()),
    ("transformer" , transformer),
    ("model", RandomForestRegressor(random_state=42,n_jobs=-1))
])

In [20]:
len(X_train)

412698

# GridSearchCV

In [35]:
param_grid = {
    "model__n_estimators": [50,100],
    "model__max_depth" : [20,None]    
}


gs = GridSearchCV(
    estimator=full_pipeline,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    return_train_score=True,
    scoring="neg_mean_squared_log_error",
    verbose=2
)

In [37]:
gs.fit(X_train[:50000],y_train[:50000])

Fitting 5 folds for each of 4 candidates, totalling 20 fits


d:\miniconda_env\env\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_engineering', NewFeature()),
                                       ('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('numerical',
                                                                         Pipeline(steps=[('median_imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          RobustScaler())]),
                                                                         ['SalesID',
                                                                          'MachineID',
                                                                          'ModelID',
                                                                          'datasource',
                                                                          'auctioneerID',
                                                                          'YearMade',
                                                                          'MachineHoursCurr...
                                                                         Pipeline(steps=[('unknown_imputer',
                                                                                          SimpleImputer(fill_value='unknown',
                                                                                                        strategy='constant')),
                                                                                         ('encoder',
                                                                                          OrdinalEncoder())]),
                                                                         ['UsageBand',
                                                                          'ProductSize'])])),
                                       ('model',
                                        RandomForestRegressor(n_jobs=-1,
                                                              random_state=42))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [20, None],
                         'model__n_estimators': [50, 100]},
             return_train_score=True, scoring='neg_mean_squared_log_error',
             verbose=2)

In [38]:
gs.best_estimator_

Pipeline(steps=[('feature_engineering', NewFeature()),
                ('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('median_imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['SalesID', 'MachineID',
                                                   'ModelID', 'datasource',
                                                   'auctioneerID', 'YearMade',
                                                   'MachineHoursCurrentMeter']),
                                                 ('categorical_bi...
                                                   'Hydraulics', 'Pushblock',
                                                   'Ripper', 'Scarifier',
                                                   'Tip_Control', 'Tire_Size',
                                                   'Coupler', 'Coupler_System',
                                                   'Grouser_Tracks', ...]),
                                                 ('categorical_ordinal',
                                                  Pipeline(steps=[('unknown_imputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['UsageBand',
                                                   'ProductSize'])])),
                ('model',
                 RandomForestRegressor(max_depth=20, n_jobs=-1,
                                       random_state=42))])

In [39]:
gs.best_params_

{'model__max_depth': 20, 'model__n_estimators': 100}

In [42]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_depth,param_model__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,114.501526,2.708544,8.081129,0.363400,20,50,"{'model__max_depth': 20, 'model__n_estimators'...",-0.076353,-0.060146,-0.050270,...,-0.064421,0.009860,3,-0.009577,-0.010091,-0.010334,-0.010189,-0.010195,-0.010077,0.000262
1,163.379113,2.261681,4.397320,2.089803,20,100,"{'model__max_depth': 20, 'model__n_estimators'...",-0.075720,-0.059455,-0.049731,...,-0.063675,0.009773,1,-0.009396,-0.009792,-0.010031,-0.009926,-0.009961,-0.009821,0.000227
2,119.367374,16.650036,8.937675,3.028091,None,50,"{'model__max_depth': None, 'model__n_estimator...",-0.076302,-0.060302,-0.050362,...,-0.064529,0.009855,4,-0.008274,-0.008649,-0.008499,-0.008157,-0.008155,-0.008347,0.000196
3,136.880022,12.870843,3.748476,1.449175,None,100,"{'model__max_depth': None, 'model__n_estimator...",-0.075856,-0.059661,-0.049778,...,-0.063840,0.009837,2,-0.008046,-0.008336,-0.008183,-0.007870,-0.007850,-0.008057,0.000186


In [61]:
best_model = gs.best_estimator_
best_model

d:\miniconda_env\env\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('feature_engineering', NewFeature()),
                ('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('median_imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['SalesID', 'MachineID',
                                                   'ModelID', 'datasource',
                                                   'auctioneerID', 'YearMade',
                                                   'MachineHoursCurrentMeter']),
                                                 ('categorical_bi...
                                                   'Hydraulics', 'Pushblock',
                                                   'Ripper', 'Scarifier',
                                                   'Tip_Control', 'Tire_Size',
                                                   'Coupler', 'Coupler_System',
                                                   'Grouser_Tracks', ...]),
                                                 ('categorical_ordinal',
                                                  Pipeline(steps=[('unknown_imputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['UsageBand',
                                                   'ProductSize'])])),
                ('model',
                 RandomForestRegressor(max_depth=20, n_jobs=-1,
                                       random_state=42))])

In [57]:
pd.DataFrame(best_model.named_steps["model"].feature_importances_).sort_values(by=0,ascending=True)

,0
3,0.000000e+00
153,8.300602e-10
150,2.475648e-06
136,2.503255e-06
28,3.530546e-06
...,...
186,8.196316e-02
0,8.444826e-02
157,1.280320e-01
2,1.978495e-01


In [62]:
best_model.fit(X_train,y_train)

d:\miniconda_env\env\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('feature_engineering', NewFeature()),
                ('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('median_imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['SalesID', 'MachineID',
                                                   'ModelID', 'datasource',
                                                   'auctioneerID', 'YearMade',
                                                   'MachineHoursCurrentMeter']),
                                                 ('categorical_bi...
                                                   'Hydraulics', 'Pushblock',
                                                   'Ripper', 'Scarifier',
                                                   'Tip_Control', 'Tire_Size',
                                                   'Coupler', 'Coupler_System',
                                                   'Grouser_Tracks', ...]),
                                                 ('categorical_ordinal',
                                                  Pipeline(steps=[('unknown_imputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['UsageBand',
                                                   'ProductSize'])])),
                ('model',
                 RandomForestRegressor(max_depth=20, n_jobs=-1,
                                       random_state=42))])

In [70]:
y_pred = best_model.predict(test)

In [69]:
y_true = test_solution["SalePrice"]

In [74]:
root_mean_squared_log_error(y_true,y_pred)

0.1289927240549204